In [7]:
conda install openpyxl

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import datetime as dt
import numpy as np
import glob
import openpyxl

## Eolicas

In [ ]:
rebelde =pd.read_csv('/Users/ctalamilla/Desktop/TacaTaca/data/Eólica 2/csv/20682654.csv', skiprows=0, sep='\t').set_axis(['fecha','hora','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
rebelde

In [55]:
rebelde['fecha'] = rebelde['fecha']+ ' ' + rebelde['hora']
rebelde

,fecha,hora,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,03/13/20 08:21:07,08:21:07,2.412,3.600,33.7
1,03/13/20 08:22:07,08:22:07,1.188,2.412,35.1
2,03/13/20 08:23:07,08:23:07,0.000,1.188,35.1
3,03/13/20 08:24:07,08:24:07,0.000,0.000,35.1
4,03/13/20 08:25:07,08:25:07,0.000,1.188,35.1
...,...,...,...,...,...
59417,04/23/20 14:38:07,14:38:07,16.848,20.448,332.7
59418,04/23/20 14:39:07,14:39:07,15.624,20.448,332.7
59419,04/23/20 14:40:07,14:40:07,18.036,22.860,351.0
59420,04/23/20 14:41:07,14:41:07,10.836,16.848,338.3


In [56]:
rebelde = rebelde.drop(['hora'], axis=1)
rebelde

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,03/13/20 08:21:07,2.412,3.600,33.7
1,03/13/20 08:22:07,1.188,2.412,35.1
2,03/13/20 08:23:07,0.000,1.188,35.1
3,03/13/20 08:24:07,0.000,0.000,35.1
4,03/13/20 08:25:07,0.000,1.188,35.1
...,...,...,...,...
59417,04/23/20 14:38:07,16.848,20.448,332.7
59418,04/23/20 14:39:07,15.624,20.448,332.7
59419,04/23/20 14:40:07,18.036,22.860,351.0
59420,04/23/20 14:41:07,10.836,16.848,338.3


In [57]:
rebelde['fecha'] = pd.to_datetime(rebelde.fecha)
rebelde

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,2020-03-13 08:21:07,2.412,3.600,33.7
1,2020-03-13 08:22:07,1.188,2.412,35.1
2,2020-03-13 08:23:07,0.000,1.188,35.1
3,2020-03-13 08:24:07,0.000,0.000,35.1
4,2020-03-13 08:25:07,0.000,1.188,35.1
...,...,...,...,...
59417,2020-04-23 14:38:07,16.848,20.448,332.7
59418,2020-04-23 14:39:07,15.624,20.448,332.7
59419,2020-04-23 14:40:07,18.036,22.860,351.0
59420,2020-04-23 14:41:07,10.836,16.848,338.3


In [52]:
rebelde.to_csv('../data/Eólica 2/all_files/20682654_fix.csv', sep=',')

### Eolica1

In [7]:
archivosAll = np.array(glob.glob('../data/Eólica 1/all_files_dmy/*'))
rutas = pd.DataFrame({'files': archivosAll})
rutas

,files
0,../data/Eólica 1/all_files_dmy/04-E1_14-07-22_...
1,../data/Eólica 1/all_files_dmy/06-E1_17-03-202...
2,../data/Eólica 1/all_files_dmy/03-E1 19-11-19_...


In [8]:
def cleanEolica(path, estacion, dmy=True):
        archivosAll = np.array(glob.glob(path +'/*'))
        rutas = pd.DataFrame({'files': archivosAll})

        eolica1 = []
        for index, value in rutas.files.str.contains('csv').items():
            if (value== True):
                #print(rutas.files[index])
                file_readed = pd.read_csv(rutas.files[index], skiprows=1, usecols=[0,1,2,3,4]).set_axis(['idm','fecha','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
                file_readed['Dir_viento_cuad']=pd.cut(file_readed['Dir_viento_grd'], bins=[0, 11.25, 33.75, 56.25, 78.75, 101.25, 123.75, 146.25,
                                                168.75, 191.25, 213.75, 236.25, 258.75, 281.25, 303.75, 326.25, 360],
                       labels=['N','NNE','NE','ENE','E','ESE','SE','SSE', 'S','SSW','SW', 'WSW', 'W', 'WNW','NW', 'NNW'])
                file_readed['fecha'] = file_readed['fecha'].apply(lambda x: pd.to_datetime(x, dayfirst=dmy))#pd.to_datetime(file_readed['fecha'] )

                file_readed['year']= file_readed['fecha'].apply(lambda x: x.year)
                file_readed['month']= file_readed['fecha'].apply(lambda x: x.month)
                file_readed['day']= file_readed['fecha'].apply(lambda x: x.day)
                file_readed['estacion']= estacion
                file_readed['file_from'] = rutas.files[index]
                eolica1.append(file_readed)
                #print(file_readed)
            # else:
            #     #print(rutas.files[index])
            #     file_readed = pd.read_excel(rutas.files[index], skiprows=1, usecols=[1,2,3,4]).set_axis(['fecha','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
            #     file_readed['fecha'] = pd.to_datetime(file_readed['fecha'] )
            #     file_readed['estacion']= estacion
            #     eolica1.append(file_readed)
            #     #print(file_readed)
        eolica2 = pd.concat(eolica1).drop_duplicates(['fecha']).reset_index(drop=True)

        return eolica2

In [16]:
EstacionEolica_dmy = cleanEolica('../data/Eólica 1/all_files_dmy', 'Eolica1', dmy=True)
EstacionEolica_dmy

,idm,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd,Dir_viento_cuad,year,month,day,estacion,file_from
0,1,2022-05-31 09:53:59,43.272,51.696,331.3,NNW,2022,5,31,Eolica1,../data/Eólica 1/all_files_dmy/04-E1_14-07-22_...
1,2,2022-05-31 09:58:59,43.272,60.120,336.9,NNW,2022,5,31,Eolica1,../data/Eólica 1/all_files_dmy/04-E1_14-07-22_...
2,3,2022-05-31 10:03:59,40.860,62.532,349.6,NNW,2022,5,31,Eolica1,../data/Eólica 1/all_files_dmy/04-E1_14-07-22_...
3,4,2022-05-31 10:08:59,44.496,57.708,341.2,NNW,2022,5,31,Eolica1,../data/Eólica 1/all_files_dmy/04-E1_14-07-22_...
4,5,2022-05-31 10:13:59,44.496,64.908,339.7,NNW,2022,5,31,Eolica1,../data/Eólica 1/all_files_dmy/04-E1_14-07-22_...
...,...,...,...,...,...,...,...,...,...,...,...
35861,23209,2019-11-19 13:00:28,4.824,7.200,141.8,SE,2019,11,19,Eolica1,../data/Eólica 1/all_files_dmy/03-E1 19-11-19_...
35862,23210,2019-11-19 13:01:28,8.424,9.612,122.1,ESE,2019,11,19,Eolica1,../data/Eólica 1/all_files_dmy/03-E1 19-11-19_...
35863,23211,2019-11-19 13:02:28,6.012,8.424,87.0,E,2019,11,19,Eolica1,../data/Eólica 1/all_files_dmy/03-E1 19-11-19_...
35864,23212,2019-11-19 13:03:28,8.424,15.624,85.6,E,2019,11,19,Eolica1,../data/Eólica 1/all_files_dmy/03-E1 19-11-19_...


In [17]:
EstacionEolica_dmy.duplicated(['fecha']).sum()

0

In [18]:
EstacionEolica_dmy['file_from'].value_counts()

../data/Eólica 1/all_files_dmy/03-E1 19-11-19_excel.csv    23213
../data/Eólica 1/all_files_dmy/04-E1_14-07-22_excel.csv    12653
Name: file_from, dtype: int64

In [19]:
EstacionEolica_dmy.groupby(['year','month'])['file_from'].count()

year  month
2019  11       23213
2022  5          170
      6         8640
      7         3843
Name: file_from, dtype: int64

In [20]:
EstacionEolica_mdy = cleanEolica('../data/Eólica 1/all_files_mdy', 'Eolica1', dmy=False)

In [21]:
EstacionEolica_mdy.groupby(['year','month'])['file_from'].count()

year  month
2019  11           1
2020  3        26828
      4        32594
2021  6        21229
      10       13355
      11       60480
      12       36480
2022  1        35510
      2        40320
      3        28237
      4         5643
Name: file_from, dtype: int64

In [23]:
EstacionEolica1 = pd.concat([EstacionEolica_mdy, EstacionEolica_dmy]).drop_duplicates(['fecha']).reset_index(drop=True)
EstacionEolica1

,idm,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd,Dir_viento_cuad,year,month,day,estacion,file_from
0,1,2021-06-08 16:34:05,28.872,42.084,206.4,SSW,2021,6,8,Eolica1,../data/Eólica 1/all_files_mdy/E1_22-10-2021.csv
1,2,2021-06-08 16:35:05,32.472,39.672,210.6,SSW,2021,6,8,Eolica1,../data/Eólica 1/all_files_mdy/E1_22-10-2021.csv
2,3,2021-06-08 16:36:05,27.648,36.072,226.0,SW,2021,6,8,Eolica1,../data/Eólica 1/all_files_mdy/E1_22-10-2021.csv
3,4,2021-06-08 16:37:05,30.060,38.484,219.0,SW,2021,6,8,Eolica1,../data/Eólica 1/all_files_mdy/E1_22-10-2021.csv
4,5,2021-06-08 16:38:05,24.048,27.648,219.0,SW,2021,6,8,Eolica1,../data/Eólica 1/all_files_mdy/E1_22-10-2021.csv
...,...,...,...,...,...,...,...,...,...,...,...
336537,23209,2019-11-19 13:00:28,4.824,7.200,141.8,SE,2019,11,19,Eolica1,../data/Eólica 1/all_files_dmy/03-E1 19-11-19_...
336538,23210,2019-11-19 13:01:28,8.424,9.612,122.1,ESE,2019,11,19,Eolica1,../data/Eólica 1/all_files_dmy/03-E1 19-11-19_...
336539,23211,2019-11-19 13:02:28,6.012,8.424,87.0,E,2019,11,19,Eolica1,../data/Eólica 1/all_files_dmy/03-E1 19-11-19_...
336540,23212,2019-11-19 13:03:28,8.424,15.624,85.6,E,2019,11,19,Eolica1,../data/Eólica 1/all_files_dmy/03-E1 19-11-19_...


In [24]:
EstacionEolica1.groupby(['year','month'])['file_from'].count()

year  month
2019  11       23213
2020  3        26828
      4        32594
2021  6        21229
      10       13355
      11       60480
      12       36480
2022  1        35510
      2        40320
      3        28237
      4         5643
      5          170
      6         8640
      7         3843
Name: file_from, dtype: int64

### Eolica2

In [25]:
EstacionEolica2_dmy = cleanEolica('../data/Eólica 2/all_files_dmy', 'Eolica1', dmy=True)
EstacionEolica2_dmy

,idm,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd,Dir_viento_cuad,year,month,day,estacion,file_from
0,1,2019-03-11 09:38:13,83.160,25.380,205.0,SSW,2019,3,11,Eolica1,../data/Eólica 2/all_files_dmy/20682650.csv
1,1,2019-11-03 09:38:13,299.376,91.368,205.0,SSW,2019,11,3,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...
2,2,2019-11-03 09:39:13,24.048,31.248,346.8,NNW,2019,11,3,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...
3,3,2019-11-03 09:40:13,25.236,34.848,346.8,NNW,2019,11,3,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...
4,4,2019-11-03 09:41:13,30.060,36.072,339.7,NNW,2019,11,3,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...
...,...,...,...,...,...,...,...,...,...,...,...
23227,23227,2019-11-19 12:44:13,7.200,12.024,123.5,ESE,2019,11,19,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...
23228,23228,2019-11-19 12:45:13,12.024,16.848,137.6,SE,2019,11,19,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...
23229,23229,2019-11-19 12:46:13,6.012,8.424,190.9,S,2019,11,19,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...
23230,23230,2019-11-19 12:47:13,2.412,7.200,171.3,S,2019,11,19,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...


In [26]:
EstacionEolica2_dmy.duplicated(['fecha']).sum()

0

In [29]:
EstacionEolica2_dmy.groupby(['year', 'month'])['fecha'].count()

year  month
2019  3            1
      11       23231
Name: fecha, dtype: int64

In [31]:
EstacionEolica2_mdy = cleanEolica('../data/Eólica 2/all_files_mdy', 'Eolica1', dmy=False)
EstacionEolica2_mdy

,idm,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd,Dir_viento_cuad,year,month,day,estacion,file_from
0,1,2022-03-17 16:48:48,30.060,43.272,158.6,SSE,2022.0,3.0,17.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_29-032022.csv
1,2,2022-03-17 16:53:48,31.248,49.284,160.0,SSE,2022.0,3.0,17.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_29-032022.csv
2,3,2022-03-17 16:58:48,32.472,46.872,165.7,SSE,2022.0,3.0,17.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_29-032022.csv
3,4,2022-03-17 17:03:48,32.472,48.096,162.9,SSE,2022.0,3.0,17.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_29-032022.csv
4,5,2022-03-17 17:08:48,15.624,24.048,162.9,SSE,2022.0,3.0,17.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_29-032022.csv
...,...,...,...,...,...,...,...,...,...,...,...
218216,110221,2022-01-07 07:58:27,-888.880,-888.880,-888.9,NaN,2022.0,1.0,7.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_07-01.csv
218217,110222,2022-01-07 07:59:27,-888.880,-888.880,-888.9,NaN,2022.0,1.0,7.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_07-01.csv
218218,110223,2022-01-07 08:00:27,-888.880,-888.880,-888.9,NaN,2022.0,1.0,7.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_07-01.csv
218219,110224,2022-01-07 08:01:27,-888.880,-888.880,-888.9,NaN,2022.0,1.0,7.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_07-01.csv


In [32]:
EstacionEolica2_mdy.groupby(['year', 'month'])['fecha'].count()

year    month
2020.0  3.0      26858
        4.0      32563
2021.0  6.0      21107
        10.0     13262
        11.0     43200
        12.0     62352
2022.0  1.0       9123
        3.0       4119
        4.0       5636
Name: fecha, dtype: int64

In [33]:
EstacionEolica2 = pd.concat([EstacionEolica2_mdy, EstacionEolica2_dmy]).drop_duplicates(['fecha']).reset_index(drop=True)
EstacionEolica2

,idm,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd,Dir_viento_cuad,year,month,day,estacion,file_from
0,1,2022-03-17 16:48:48,30.060,43.272,158.6,SSE,2022.0,3.0,17.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_29-032022.csv
1,2,2022-03-17 16:53:48,31.248,49.284,160.0,SSE,2022.0,3.0,17.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_29-032022.csv
2,3,2022-03-17 16:58:48,32.472,46.872,165.7,SSE,2022.0,3.0,17.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_29-032022.csv
3,4,2022-03-17 17:03:48,32.472,48.096,162.9,SSE,2022.0,3.0,17.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_29-032022.csv
4,5,2022-03-17 17:08:48,15.624,24.048,162.9,SSE,2022.0,3.0,17.0,Eolica1,../data/Eólica 2/all_files_mdy/E2_29-032022.csv
...,...,...,...,...,...,...,...,...,...,...,...
241448,23227,2019-11-19 12:44:13,7.200,12.024,123.5,ESE,2019.0,11.0,19.0,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...
241449,23228,2019-11-19 12:45:13,12.024,16.848,137.6,SE,2019.0,11.0,19.0,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...
241450,23229,2019-11-19 12:46:13,6.012,8.424,190.9,S,2019.0,11.0,19.0,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...
241451,23230,2019-11-19 12:47:13,2.412,7.200,171.3,S,2019.0,11.0,19.0,Eolica1,../data/Eólica 2/all_files_dmy/E2 19-11-19_exc...


In [34]:
EstacionEolica2.groupby(['year', 'month'])['fecha'].count()

year    month
2019.0  3.0          1
        11.0     23231
2020.0  3.0      26858
        4.0      32563
2021.0  6.0      21107
        10.0     13262
        11.0     43200
        12.0     62352
2022.0  1.0       9123
        3.0       4119
        4.0       5636
Name: fecha, dtype: int64

In [426]:
EstacionEolica1.to_csv('Eolica1_cleaned.csv', sep=',')
EstacionEolica2.to_csv('Eolica2_cleaned.csv', sep=',')

#### Direccion Viento en grados y Cuadrantes

N 348.75 - 11.25

NNE 11.25 - 33.75

NE 33.75 - 56.25

ENE 56.25 - 78.75

E 78.75 - 101.25

ESE 101.25 - 123.75

SE 123.75 - 146.25

SSE 146.25 - 168.75

S 168.75 - 191.25

SSW 191.25 - 213.75

SW 213.75 - 236.25

WSW 236.25 - 258.75

W 258.75 - 281.25

WNW 281.25 - 303.75

NW 303.75 - 326.25

NNW 326.25 - 348.75

## Davis 

In [349]:
def cleanDavis(ruta, estacion):
        archivosAll = np.array(glob.glob(ruta +'/*'))
        rutas = pd.DataFrame({'files': archivosAll})
        rutas

        davis = []
        for index, value in rutas.files.str.contains('csv').items():
                #print(rutas.files[index])
                colNames = ['fecha', 
                            'hora', 
                            'tempAmbiente-c', 
                            'tempMaxima-c', 
                            'tempMinima-c',
                            'humRelativa-%', 
                            'puntoRocio-c', 
                            'velViento-mps',
                            'dirViento-cuad', 
                            'relViento',
                            'velVientoMaxima-mps', 
                            'dirVientoPredominante-cuad', 
                            'sensTermina-c',
                            'indiceCalor', 
                            'indiceTHW',
                            'indiceTHSW', 
                            'presionATM',
                            'precipitacion-mm', 
                            'intPrecipitacion-mmh',
                            'radiacionSolar',
                            'energiaSolar', 
                            'radiacionSolarMaxima', 
                            'heatD-D', 
                            'coolD-D',
                            'tempInterior-c',
                            'humRelativaInterior-%', 
                            'puntoRocioInterior-c',
                            'indiceCalorInterior',
                            'evapotranspiracion-mm',
                            'windSamp', 
                            'windTx',
                            'ISSRecept',
                            'arcInt']
                file_readed = pd.read_csv(rutas.files[index], skiprows=2, sep=';', low_memory=False).set_axis(colNames, axis='columns')
                file_readed['Estacion'] = estacion
                file_readed['file_from'] = rutas.files[index]
                file_readed['fechaHora'] = pd.to_datetime(file_readed['fecha']+ ' ' + file_readed['hora'])
                
                davis.append(file_readed)
                #print(file_readed)
        return pd.concat(davis).drop_duplicates(['fecha', 'hora']).reset_index(drop=True)

In [154]:
Davis_TacaTaca_cleaned = cleanDavis('../data/EM_Taca Taca/all_files', 'Estacion Taca Taca')

In [155]:
Davis_TacaTaca_cleaned

,fecha,hora,tempAmbiente-c,tempMaxima-c,tempMinima-c,humRelativa-%,puntoRocio-c,velViento-mps,dirViento-cuad,relViento,...,humRelativaInterior-%,puntoRocioInterior-c,indiceCalorInterior,evapotranspiracion-mm,windSamp,windTx,ISSRecept,arcInt,Estacion,fechaHora
0,30/9/11,10:00,10.4,10.4,9.3,15,-15.2,8.0,SE,2.01,...,14,-14.2,10.5,0.3,351,1,100.0,15,Estacion Taca Taca,2011-09-30 10:00:00
1,30/9/11,10:15,10.7,10.8,10.4,14,-15.8,8.0,SSE,2.01,...,14,-14.4,10.3,0,351,1,100.0,15,Estacion Taca Taca,2011-09-30 10:15:00
2,30/9/11,10:30,10.8,11,10.7,13,-16.6,11.3,SSE,2.82,...,14,-14.5,10.3,0,352,1,100.0,15,Estacion Taca Taca,2011-09-30 10:30:00
3,30/9/11,10:45,10.7,10.9,10.7,14,-15.8,11.3,SSE,2.82,...,14,-14.6,10.2,0,351,1,100.0,15,Estacion Taca Taca,2011-09-30 10:45:00
4,30/9/11,11:00,11.6,11.6,10.7,13,-15.9,9.7,SSE,2.41,...,13,-15.2,10.4,0.48,351,1,100.0,15,Estacion Taca Taca,2011-09-30 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253890,30/4/21,4:00 a.m.,0.5,1.2,-0.4,19,-20.5,4.8,NO,9.66,...,30,-17.6,-2.9,0.13,2805,1,100.0,120,Estacion Taca Taca,2021-04-30 04:00:00
253891,30/4/21,6:00 a.m.,-1.5,-0.4,-2.5,24,-19.5,8.0,NO,16.09,...,32,-17.8,-4.0,0.13,2807,1,100.0,120,Estacion Taca Taca,2021-04-30 06:00:00
253892,30/4/21,8:00 a.m.,-2.6,-1.0,-3.6,24,-20.4,8.0,ONO,16.09,...,32,-18.8,-5.1,0.10,2804,1,100.0,120,Estacion Taca Taca,2021-04-30 08:00:00
253893,30/4/21,10:00 a.m.,-1.1,2.9,-4.1,18,-22.4,3.2,N,6.44,...,33,-17.9,-4.4,0.18,2806,1,100.0,120,Estacion Taca Taca,2021-04-30 10:00:00


In [156]:
Davis_Vendaval_cleaned=cleanDavis('../data/EM_Vendaval/all_files', 'Estacion Vendaval')

In [157]:
Davis_Vendaval_cleaned

,fecha,hora,tempAmbiente-c,tempMaxima-c,tempMinima-c,humRelativa-%,puntoRocio-c,velViento-mps,dirViento-cuad,relViento,...,humRelativaInterior-%,puntoRocioInterior-c,indiceCalorInterior,evapotranspiracion-mm,windSamp,windTx,ISSRecept,arcInt,Estacion,fechaHora
0,6/6/21,14:00,1.9,2.1,0.2,19,-19.3,27.4,WNW,54.72,...,19,-15.4,5.6,0.84,2801,1,100.0,120,Estacion Vendaval,2021-06-06 14:00:00
1,6/6/21,16:00,1.6,2.4,1.3,21,-18.4,20.9,NW,41.84,...,17,-14.6,8.0,0.74,2806,1,100.0,120,Estacion Vendaval,2021-06-06 16:00:00
2,6/6/21,18:00,0.2,1.8,0.2,23,-18.6,37.0,WNW,74.03,...,17,-18.6,3.3,0.61,2802,1,100.0,120,Estacion Vendaval,2021-06-06 18:00:00
3,6/6/21,20:00,-3.1,0.2,-3.1,29,-18.6,37.0,WNW,74.03,...,19,-24.6,-4.9,0.33,2802,1,100.0,120,Estacion Vendaval,2021-06-06 20:00:00
4,6/6/21,22:00,-4.2,-3.0,-4.2,31,-18.8,17.7,N,35.41,...,22,-25.0,-7.3,0.15,2807,1,100.0,120,Estacion Vendaval,2021-06-06 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4295,30/5/22,10:00,-3.7,-3.7,-7.6,45,-13.9,25.7,S,51.50,...,38,-16.3,-4.4,0.30,2806,1,100.0,120,Estacion Vendaval,2022-05-30 10:00:00
4296,30/5/22,12:00,-0.8,-0.8,-3.8,28,-17.1,41.8,S,83.69,...,32,-14.6,-0.4,0.74,2805,1,100.0,120,Estacion Vendaval,2022-05-30 12:00:00
4297,30/5/22,14:00,0.5,0.9,-1.0,22,-18.8,54.7,S,109.44,...,24,-14.1,3.8,1.07,2802,1,100.0,120,Estacion Vendaval,2022-05-30 14:00:00
4298,30/5/22,16:00,0.7,1.6,0.4,19,-20.3,56.3,S,112.65,...,21,-16.4,3.0,0.94,2807,1,100.0,120,Estacion Vendaval,2022-05-30 16:00:00


In [158]:
Davis_TacaTaca_cleaned.to_csv('Davis_TacaTaca.csv', sep=',')
Davis_Vendaval_cleaned.to_csv('DavisVendaval.csv', sep=',')

## Campbell

In [305]:
columnas = pd.read_csv('../data/EM_Caipe/all_files/CR300Series_DATA_05M.csv', 
                       names=list(range(0,38)),
                       skiprows=1,sep=',', low_memory= False, nrows=3)
columnas

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,TIMESTAMP,RECORD,BattV_Avg,LoggerTemp_C_Avg,VelViento_ms_Min,VelViento_ms_Avg,DirViento_Grad,Sigma_Theta,VelViento_ms_Max,Rafaga_ms_Max,...,Q,TCDT,AltNieve_m,RadSolar_Wm2_Min,RadSolar_Wm2_Avg,RadSolar_Wm2_Max,RadSolar_MJ_Tot,TPRocio_C_Avg,TBHumedo_C_Avg,IndiceCalor_C_Avg
1,TS,RN,Volts,Grad_Celsius,m/s,m/s,Grados_Sexag,Deg,m/s,NaN,...,NaN,NaN,meters,Watts/m^2,Watts/m^2,Watts/m^2,MJ/m^2,Grad_Celsius,Grad_Celsius,Grad_Celsius
2,NaN,NaN,Avg,Avg,Min,WVc,WVc,WVc,Max,Max,...,Smp,Smp,Smp,Min,Avg,Max,Tot,Avg,Avg,Avg


In [310]:
columnasCampell = ['-'.join([str(i) for i in columnas.iloc[:, i].values]) for i in columnas.columns]
columnasCampell


['TIMESTAMP-TS-nan',
 'RECORD-RN-nan',
 'BattV_Avg-Volts-Avg',
 'LoggerTemp_C_Avg-Grad_Celsius-Avg',
 'VelViento_ms_Min-m/s-Min',
 'VelViento_ms_Avg-m/s-WVc',
 'DirViento_Grad-Grados_Sexag-WVc',
 'Sigma_Theta-Deg-WVc',
 'VelViento_ms_Max-m/s-Max',
 'Rafaga_ms_Max-nan-Max',
 'VelViento_KPH_Min-nan-Min',
 'VelViento_KPH_Avg-nan-Avg',
 'VelViento_KPH_Max-nan-Max',
 'TempAmb_C_Min-Grad_Celsius-Min',
 'TempAmb_C_Avg-Grad_Celsius-Avg',
 'TempAmb_C_Max-Grad_Celsius-Max',
 'HRelativa_Min-%-Min',
 'HRelativa_Avg-%-Avg',
 'HRelativa_Max-%-Max',
 'HRelativa-%-Smp',
 'PBar_mbar_Min-mbar-Min',
 'PBar_mbar_Avg-mbar-Avg',
 'PBar_mbar_Max-mbar-Max',
 'Precip_mm_Tot-mm-Tot',
 'RadNeta_Wm2_Min-Watts/m^2-Min',
 'RadNeta_Wm2_Avg-Watts/m^2-Avg',
 'RadNeta_Wm2_Max-Watts/m^2-Max',
 'DT-nan-Smp',
 'Q-nan-Smp',
 'TCDT-nan-Smp',
 'AltNieve_m-meters-Smp',
 'RadSolar_Wm2_Min-Watts/m^2-Min',
 'RadSolar_Wm2_Avg-Watts/m^2-Avg',
 'RadSolar_Wm2_Max-Watts/m^2-Max',
 'RadSolar_MJ_Tot-MJ/m^2-Tot',
 'TPRocio_C_Avg-Grad_Ce

In [338]:
prueba = pd.read_csv('../data/EM_Caipe/all_files/CR300Series_DATA_05M.csv', 
                       names=columnasCampell, skiprows=4, low_memory=False)

In [340]:
pd.to_datetime(prueba['TIMESTAMP-TS-nan'])

0        2019-08-06 14:00:00
1        2019-08-06 14:05:00
2        2019-08-06 14:10:00
3        2019-08-06 14:15:00
4        2019-08-06 14:20:00
                 ...        
284209   2022-04-19 10:25:00
284210   2022-04-19 10:30:00
284211   2022-04-19 10:35:00
284212   2022-04-19 10:40:00
284213   2022-04-19 10:45:00
Name: TIMESTAMP-TS-nan, Length: 284214, dtype: datetime64[ns]

In [387]:
def cleanCampbell_5m_24h(ruta, estacion):
        archivosAll = np.array(glob.glob(ruta +'/*'))
        rutas = pd.DataFrame({'files': archivosAll})
        rutas

        campbell = []
        for index, value in rutas.files.str.contains('csv').items():
                #print(rutas.files[index])
                
                file_readed = pd.read_csv(rutas.files[index], names=columnasCampell, skiprows=4, sep=',', low_memory=False)
                file_readed['Estacion'] = estacion
                file_readed['fechaHora'] = pd.to_datetime(file_readed['TIMESTAMP-TS-nan'])
                file_readed['file_from'] = rutas.files[index]
                campbell.append(file_readed)
                #print(file_readed)
        return pd.concat(campbell).drop_duplicates(['fechaHora']).reset_index(drop=True)

In [388]:
cleanCampbell_5m_24h('../data/EM_Caipe/all_files/5m', 'Caipe')


,TIMESTAMP-TS-nan,RECORD-RN-nan,BattV_Avg-Volts-Avg,LoggerTemp_C_Avg-Grad_Celsius-Avg,VelViento_ms_Min-m/s-Min,VelViento_ms_Avg-m/s-WVc,DirViento_Grad-Grados_Sexag-WVc,Sigma_Theta-Deg-WVc,VelViento_ms_Max-m/s-Max,Rafaga_ms_Max-nan-Max,...,RadSolar_Wm2_Min-Watts/m^2-Min,RadSolar_Wm2_Avg-Watts/m^2-Avg,RadSolar_Wm2_Max-Watts/m^2-Max,RadSolar_MJ_Tot-MJ/m^2-Tot,TPRocio_C_Avg-Grad_Celsius-Avg,TBHumedo_C_Avg-Grad_Celsius-Avg,IndiceCalor_C_Avg-Grad_Celsius-Avg,Estacion,fechaHora,file_from
0,2019-08-06 14:00:00,0,13.91,14.170,0,5.964,302.6,14.360,9.2,8.650,...,828,828,830,0.2982346,-29.88,-0.510,12.360,Caipe,2019-08-06 14:00:00,../data/EM_Caipe/all_files/5m/CR300Series_DATA...
1,2019-08-06 14:05:00,1,13.91,14.030,2.266,6.251,310.2,11.700,10.7,10.330,...,827,827,828,0.4962566,-29.57,-0.413,12.520,Caipe,2019-08-06 14:05:00,../data/EM_Caipe/all_files/5m/CR300Series_DATA...
2,2019-08-06 14:10:00,2,13.91,14.060,3.299,6.189,318.8,9.130,8.53,8.090,...,824,826,827,0.4954809,-29.33,-0.277,12.770,Caipe,2019-08-06 14:10:00,../data/EM_Caipe/all_files/5m/CR300Series_DATA...
3,2019-08-06 14:15:00,3,13.91,14.050,3.065,6.442,313.5,13.500,8.63,8.530,...,822,824,825,0.4942019,-29.45,-0.198,12.940,Caipe,2019-08-06 14:15:00,../data/EM_Caipe/all_files/5m/CR300Series_DATA...
4,2019-08-06 14:20:00,4,13.91,13.630,3.898,6.561,323.6,10.260,9.33,8.670,...,819,821,822,0.492659,-43.73,-2.408,9.590,Caipe,2019-08-06 14:20:00,../data/EM_Caipe/all_files/5m/CR300Series_DATA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284206,2022-04-19 10:25:00,284209,13.99,5.523,0,7.253,332.3,7.820,9.33,9.170,...,558.5,566,573.7,0.294318,-27.56,-7.173,-0.093,Caipe,2022-04-19 10:25:00,../data/EM_Caipe/all_files/5m/CR300Series_DATA...
284207,2022-04-19 10:30:00,284210,13.97,5.621,5.631,7.173,329.5,4.999,8.23,7.908,...,573.6,581.1,589.3,0.3486631,-27.58,-7.129,-0.017,Caipe,2022-04-19 10:30:00,../data/EM_Caipe/all_files/5m/CR300Series_DATA...
284208,2022-04-19 10:35:00,284211,13.95,5.711,5.931,6.973,334.4,6.664,8.36,8.220,...,589.5,597.7,605.9,0.3586366,-27.50,-7.014,0.164,Caipe,2022-04-19 10:35:00,../data/EM_Caipe/all_files/5m/CR300Series_DATA...
284209,2022-04-19 10:40:00,284212,13.94,5.927,5.165,6.38,327.5,6.102,7.664,7.652,...,606.1,614,621.5,0.3683733,-27.74,-6.843,0.481,Caipe,2022-04-19 10:40:00,../data/EM_Caipe/all_files/5m/CR300Series_DATA...


In [389]:
cleanCampbell_5m_24h('../data/EM_Caipe/all_files/24h', 'Caipe')

,TIMESTAMP-TS-nan,RECORD-RN-nan,BattV_Avg-Volts-Avg,LoggerTemp_C_Avg-Grad_Celsius-Avg,VelViento_ms_Min-m/s-Min,VelViento_ms_Avg-m/s-WVc,DirViento_Grad-Grados_Sexag-WVc,Sigma_Theta-Deg-WVc,VelViento_ms_Max-m/s-Max,Rafaga_ms_Max-nan-Max,...,RadSolar_Wm2_Min-Watts/m^2-Min,RadSolar_Wm2_Avg-Watts/m^2-Avg,RadSolar_Wm2_Max-Watts/m^2-Max,RadSolar_MJ_Tot-MJ/m^2-Tot,TPRocio_C_Avg-Grad_Celsius-Avg,TBHumedo_C_Avg-Grad_Celsius-Avg,IndiceCalor_C_Avg-Grad_Celsius-Avg,Estacion,fechaHora,file_from
0,2019-08-07 00:00:00,0,13.31,7.743,0,7.071,289.7,18.70,16.49,15.46,...,0,268.4,830,19.42136,-39.25,-5.743,3.522,Caipe,2019-08-07,../data/EM_Caipe/all_files/24h/CR300Series_DAT...
1,2019-08-08 00:00:00,1,13.14,3.369,0.033,8.49,272.5,25.92,20.13,19.77,...,NAN,NAN,NAN,NAN,-33.62,-7.518,0.100,Caipe,2019-08-08,../data/EM_Caipe/all_files/24h/CR300Series_DAT...
2,2019-08-09 00:00:00,2,13.12,2.303,6.198,15.25,305.5,11.12,31.29,29.02,...,0,222.4,839,38.43308,-24.38,-8.280,-2.358,Caipe,2019-08-09,../data/EM_Caipe/all_files/24h/CR300Series_DAT...
3,2019-08-10 00:00:00,3,13.15,0.564,0,9.39,272.1,32.93,22.09,21.44,...,0,164.1,714.2,9.884465,-26.80,-10.010,-4.652,Caipe,2019-08-10,../data/EM_Caipe/all_files/24h/CR300Series_DAT...
4,2019-08-11 00:00:00,4,13.11,-0.176,0,7.067,290.5,46.65,19.09,18.67,...,0,239.7,844,41.41201,-33.79,-10.080,-4.058,Caipe,2019-08-11,../data/EM_Caipe/all_files/24h/CR300Series_DAT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,2022-04-15 00:00:00,982,13.14,2.161,0,8.84,335.8,23.54,23.92,22.49,...,0,269.9,925,46.64567,-30.10,-8.570,-1.989,Caipe,2022-04-15,../data/EM_Caipe/all_files/24h/CR300Series_DAT...
983,2022-04-16 00:00:00,983,13.12,0.928,0,8.17,295.7,41.07,23.52,23.29,...,0,268,920,46.31055,-26.55,-9.210,-3.529,Caipe,2022-04-16,../data/EM_Caipe/all_files/24h/CR300Series_DAT...
984,2022-04-17 00:00:00,984,13.12,0.370,0,7.738,316.4,29.69,17.99,17.76,...,0,266.3,918,46.02095,-25.72,-9.540,-4.172,Caipe,2022-04-17,../data/EM_Caipe/all_files/24h/CR300Series_DAT...
985,2022-04-18 00:00:00,985,13.14,1.096,0,4.935,280.0,52.81,15.69,15.03,...,0,265.5,914,45.87931,-29.99,-9.130,-2.833,Caipe,2022-04-18,../data/EM_Caipe/all_files/24h/CR300Series_DAT...


In [ ]:
'-'.join(col[3:6])

'RECORD-RN-nan'

## 60 m

In [382]:
columnas60m = pd.read_csv('../data/EM_Caipe/all_files/60m/CR300Series_DATA_60m.csv', 
                       names=list(range(0,40)),
                       skiprows=1,sep=',', low_memory= False, nrows=3)
columnas60m

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,TIMESTAMP,RECORD,BattV_Avg,LoggerTemp_C_Avg,VelViento_ms_Min,VelViento_ms_Avg,DirViento_Grad,Sigma_Theta,VelViento_ms_Max,Rafaga_ms_Max,...,AltNieve_m,RadSolar_Wm2_Min,RadSolar_Wm2_Avg,RadSolar_Wm2_Max,RadSolar_MJ_Tot,ETos,Rso,TPRocio_C_Avg,TBHumedo_C_Avg,IndiceCalor_C_Avg
1,TS,RN,Volts,Grad_Celsius,m/s,m/s,Grados_Sexag,Deg,m/s,NaN,...,meters,Watts/m^2,Watts/m^2,Watts/m^2,MJ/m^2,Grad_Celsius,Grad_Celsius,Grad_Celsius,Grad_Celsius,Grad_Celsius
2,NaN,NaN,Avg,Avg,Min,WVc,WVc,WVc,Max,Max,...,Smp,Min,Avg,Max,Tot,ETXs,Rso,Avg,Avg,Avg


In [383]:
columnasCampell60m = ['-'.join([str(i) for i in columnas60m.iloc[:, i].values]) for i in columnas60m.columns]
columnasCampell60m


['TIMESTAMP-TS-nan',
 'RECORD-RN-nan',
 'BattV_Avg-Volts-Avg',
 'LoggerTemp_C_Avg-Grad_Celsius-Avg',
 'VelViento_ms_Min-m/s-Min',
 'VelViento_ms_Avg-m/s-WVc',
 'DirViento_Grad-Grados_Sexag-WVc',
 'Sigma_Theta-Deg-WVc',
 'VelViento_ms_Max-m/s-Max',
 'Rafaga_ms_Max-nan-Max',
 'VelViento_KPH_Min-nan-Min',
 'VelViento_KPH_Avg-nan-Avg',
 'VelViento_KPH_Max-nan-Max',
 'TempAmb_C_Min-Grad_Celsius-Min',
 'TempAmb_C_Avg-Grad_Celsius-Avg',
 'TempAmb_C_Max-Grad_Celsius-Max',
 'HRelativa_Min-%-Min',
 'HRelativa_Avg-%-Avg',
 'HRelativa_Max-%-Max',
 'HRelativa-%-Smp',
 'PBar_mbar_Min-mbar-Min',
 'PBar_mbar_Avg-mbar-Avg',
 'PBar_mbar_Max-mbar-Max',
 'Precip_mm_Tot-mm-Tot',
 'RadNeta_Wm2_Min-Watts/m^2-Min',
 'RadNeta_Wm2_Avg-Watts/m^2-Avg',
 'RadNeta_Wm2_Max-Watts/m^2-Max',
 'DT-nan-Smp',
 'Q-nan-Smp',
 'TCDT-nan-Smp',
 'AltNieve_m-meters-Smp',
 'RadSolar_Wm2_Min-Watts/m^2-Min',
 'RadSolar_Wm2_Avg-Watts/m^2-Avg',
 'RadSolar_Wm2_Max-Watts/m^2-Max',
 'RadSolar_MJ_Tot-MJ/m^2-Tot',
 'ETos-Grad_Celsius-ETX

In [429]:
def cleanCampbell_60m(ruta, estacion):
        archivosAll = np.array(glob.glob(ruta +'/*'))
        rutas = pd.DataFrame({'files': archivosAll})
        rutas

        campbell = []
        for index, value in rutas.files.str.contains('csv').items():
                #print(rutas.files[index])
                
                file_readed = pd.read_csv(rutas.files[index], names=columnasCampell60m, skiprows=4, sep=',', low_memory=False)
                file_readed['DirViento_cuad']=pd.cut(file_readed['DirViento_Grad-Grados_Sexag-WVc'], bins=[0, 11.25, 33.75, 56.25, 78.75, 101.25, 123.75, 146.25,
                                                168.75, 191.25, 213.75, 236.25, 258.75, 281.25, 303.75, 326.25, 360],
                       labels=['N','NNE','NE','ENE','E','ESE','SE','SSE', 'S','SSW','SW', 'WSW', 'W', 'WNW','NW', 'NNW'])
                file_readed['Estacion'] = estacion
                file_readed['fechaHora'] = pd.to_datetime(file_readed['TIMESTAMP-TS-nan'])
                file_readed['file_from'] = rutas.files[index]
                campbell.append(file_readed)
                #print(file_readed)
        return pd.concat(campbell).drop_duplicates(['fechaHora']).reset_index(drop=True)

In [430]:
cleanCampbell_60m('../data/EM_Caipe/all_files/60m', 'Caipe')

,TIMESTAMP-TS-nan,RECORD-RN-nan,BattV_Avg-Volts-Avg,LoggerTemp_C_Avg-Grad_Celsius-Avg,VelViento_ms_Min-m/s-Min,VelViento_ms_Avg-m/s-WVc,DirViento_Grad-Grados_Sexag-WVc,Sigma_Theta-Deg-WVc,VelViento_ms_Max-m/s-Max,Rafaga_ms_Max-nan-Max,...,RadSolar_MJ_Tot-MJ/m^2-Tot,ETos-Grad_Celsius-ETXs,Rso-Grad_Celsius-Rso,TPRocio_C_Avg-Grad_Celsius-Avg,TBHumedo_C_Avg-Grad_Celsius-Avg,IndiceCalor_C_Avg-Grad_Celsius-Avg,DirViento_cuad,Estacion,fechaHora,file_from
0,2019-08-06 14:00:00,0,13.91,14.170,0,5.964,302.6,14.36,9.2,8.650,...,0.2982346,0.222,2.350,-29.88,-0.510,12.360,WNW,Caipe,2019-08-06 14:00:00,../data/EM_Caipe/all_files/60m/CR300Series_DAT...
1,2019-08-06 15:00:00,1,13.91,13.450,1.433,7.006,315.7,13.98,11.73,11.310,...,5.829209,0.908,2.025,-39.65,-2.350,9.470,NW,Caipe,2019-08-06 15:00:00,../data/EM_Caipe/all_files/60m/CR300Series_DAT...
2,2019-08-06 16:00:00,2,13.91,13.950,3.532,8.84,301.5,16.88,16.49,15.460,...,5.249954,0.798,1.264,-44.23,-2.808,8.840,WNW,Caipe,2019-08-06 16:00:00,../data/EM_Caipe/all_files/60m/CR300Series_DAT...
3,2019-08-06 17:00:00,3,13.90,14.270,5.265,9.73,289.5,12.57,14.53,14.210,...,4.156892,0.652,0.404,-42.48,-2.953,8.510,WNW,Caipe,2019-08-06 17:00:00,../data/EM_Caipe/all_files/60m/CR300Series_DAT...
4,2019-08-06 18:00:00,4,13.89,13.880,5.931,10,292.6,12.21,14.29,13.830,...,2.727247,0.165,0.002,-41.35,-3.458,7.532,WNW,Caipe,2019-08-06 18:00:00,../data/EM_Caipe/all_files/60m/CR300Series_DAT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23680,2022-04-19 06:00:00,23680,12.49,-3.637,0,3.039,326.9,14.04,6.831,6.641,...,0,0.014,0.004,-27.39,-10.300,-5.097,NNW,Caipe,2022-04-19 06:00:00,../data/EM_Caipe/all_files/60m/CR300Series_DAT...
23681,2022-04-19 07:00:00,23681,12.46,-3.687,0,1.95,326.7,22.74,4.731,4.353,...,0,0.033,0.450,-30.62,-10.970,-5.708,NNW,Caipe,2022-04-19 07:00:00,../data/EM_Caipe/all_files/60m/CR300Series_DAT...
23682,2022-04-19 08:00:00,23682,12.41,-4.487,0,1.659,307.8,27.98,4.098,3.920,...,0.009342839,0.03,1.361,-27.91,-10.410,-5.191,NW,Caipe,2022-04-19 08:00:00,../data/EM_Caipe/all_files/60m/CR300Series_DAT...
23683,2022-04-19 09:00:00,23683,12.76,-3.561,0,3.393,323.8,15.66,6.997,6.785,...,0.4503329,0.096,2.179,-26.04,-9.830,-4.575,NW,Caipe,2022-04-19 09:00:00,../data/EM_Caipe/all_files/60m/CR300Series_DAT...
